In [21]:
import json
import pandas as pd
import re


BASE_PATH = "../base_completa/boatos_br_corpus.json"
CATEGORIAS_PROCESSADAS_PATH = "./categorias/categorias_processadas.json"
ORIGENS_PROCESSADAS_PATH = "./origem/origens_processadas.json"

PALAVRAS_PARA_ID_PATH = "./tokens/tokenstoid.json"

In [22]:
database = pd.read_json(BASE_PATH).to_dict('records')

### Carregar dados processados

In [23]:
def loadJson(path:str):
    aux = []
    with open(path,'r') as f:
        aux = json.load(f)
        f.close()
    return aux

def normalizarDadoProcessado(dado:str,listaNormalizacao:dict) -> int:
    return listaNormalizacao[dado] if listaNormalizacao.get(dado) != None else -1

def normalizarTexto(texto:str,listaNormalizacao:list) -> list:
    palavras_text_normalizadas = []
    palavras_texto = texto.split(" ")
    for palavra in palavras_texto:
        palavras_text_normalizadas.append(str(normalizarDadoProcessado(palavra,listaNormalizacao)))
    return palavras_text_normalizadas
 

In [24]:
categoriasDict = loadJson(CATEGORIAS_PROCESSADAS_PATH)
origensDict = loadJson(ORIGENS_PROCESSADAS_PATH) 
rotulosDict = {
    'falso': 0,
    'verdade': 1,
    'meia-verdade': 2
}
words2Id = loadJson(PALAVRAS_PARA_ID_PATH)

In [25]:
for data in database:
    data['categorias'] = [normalizarDadoProcessado(x,categoriasDict) for x in data['categorias']]
    data['origem'] = normalizarDadoProcessado(data['origem'],origensDict)
    data['data-publicacao'] = pd.to_datetime(data['data-publicacao']).timestamp()
    data['rotulo'] = normalizarDadoProcessado(data['rotulo'],rotulosDict)
    data['texto-normalizado'] = " ".join(normalizarTexto(data['texto-normalizado'],words2Id))

In [27]:
with open(f"../base_processada/boatos_br_corpus_processada1_att.json",'w',encoding="utf8") as f:
    json.dump(database,f,indent=2,allow_nan=True,ensure_ascii = False,separators=(',', ':'))
    f.close()